<a href="https://colab.research.google.com/github/rypotter/depo/blob/master/rag-chatbot-using-llamaindex-groq-with-llama3-chainlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

https://blog.gopenai.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

In [ ]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

### Import Libraries

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from google.colab import userdata
# GROQ_API_KEY = userdata.get('GROQ_API_KEY')
GROQ_API_KEY = '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'
# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [7]:
!pwd
!ls -al
!ls ./Basics_of_finance_1.pdf

/content
total 1852
drwxr-xr-x 1 root root    4096 Aug 27 13:56 .
drwxr-xr-x 1 root root    4096 Aug 27 13:50 ..
-rw-r--r-- 1 root root 1879774 Aug 27 13:56 Basics_of_finance_1.pdf
drwxr-xr-x 4 root root    4096 Aug 23 13:20 .config
drwxr-xr-x 1 root root    4096 Aug 23 13:20 sample_data
./Basics_of_finance_1.pdf


In [8]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./Basics_of_finance_1.pdf"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [9]:
len(documents)

86

In [10]:
documents[4].metadata

{'page_label': '5',
 'file_name': 'Basics_of_finance_1.pdf',
 'file_path': 'Basics_of_finance_1.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-08-27',
 'last_modified_date': '2024-08-27'}

### Chunking

In [11]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [12]:
len(nodes)

86

In [13]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'Basics_of_finance_1.pdf',
 'file_path': 'Basics_of_finance_1.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-08-27',
 'last_modified_date': '2024-08-27'}

https://chunkviz.up.railway.app/

### Embedding Model

In [14]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [15]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [16]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [17]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [18]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [19]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [20]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [21]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

In [24]:
query = "Explain market bonds?"
resp = query_engine.query(query)

In [25]:
print(resp.response)

Bonds are debt instruments similar to loans, where the issuer is the debtor and the investor or buyer of the bond is the lender. The issuer promises to pay the interest and the nominal or face value of the security in the future. Bonds are traded securities, which means that the 'lender' does not have to keep the asset until maturity, they can sell it on the secondary market. The interest rate that the bond pays, also called the coupon rate, is not necessarily equal to the return expected by the investors.


https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/